# Analise de sentimentos de assédio moral

In [ ]:
# Instalar bibliotecas necessárias
!pip install nltk textblob matplotlib pandas scikit-learn requests

# Importar bibliotecas
import nltk
from textblob import TextBlob
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn import metrics

# Baixar dados de análise de sentimentos do nltk
nltk.download('punkt')

# Função para ler o arquivo CSV de treinamento a partir de uma URL
def ler_csv_url(url):
    df = pd.read_csv(url)
    return df

# Função para preparar os dados a partir de um dataframe CSV
def preparar_dados_csv(df):
    # Verifica se o arquivo tem as colunas necessárias
    if 'fala' not in df.columns or 'comportamento' not in df.columns:
        raise ValueError("O arquivo CSV não contém as colunas 'fala' e 'comportamento'.")

    textos = df['fala'].tolist()
    etiquetas = df['comportamento'].tolist()

    print(f"Número de amostras para treinamento: {len(textos)}")

    if not textos:
        print("Aviso: Nenhuma amostra de treinamento foi preparada.")

    return textos, etiquetas

# Função para treinar o modelo de classificação para detectar padrões abusivos
def treinar_modelo(textos_treinamento, etiquetas_treinamento):
    if len(textos_treinamento) == 0:
        raise ValueError("Nenhuma amostra de treinamento foi fornecida.")

    # Dividir dados em treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(textos_treinamento, etiquetas_treinamento, test_size=0.2, random_state=42)

    # Criar e treinar o modelo de NLP
    model = make_pipeline(TfidfVectorizer(), MultinomialNB())
    model.fit(X_train, y_train)

    # Avaliação do Modelo
    y_pred = model.predict(X_test)
    print("Acurácia:", metrics.accuracy_score(y_test, y_pred))
    print("Precisão:", metrics.precision_score(y_test, y_pred, average='weighted'))
    print("Recall:", metrics.recall_score(y_test, y_pred, average='weighted'))
    print("F1-score:", metrics.f1_score(y_test, y_pred, average='weighted'))

    return model

# Função para ler o texto a partir de uma URL
def ler_texto_url(url):
    response = requests.get(url)
    return response.text

# Função para avaliar o modelo em novos dados
def avaliar_modelo(modelo, url_teste):
    texto_teste = ler_texto_url(url_teste)
    if texto_teste:
        # Preparar dados para o conjunto de teste
        textos, _ = preparar_dados_csv(pd.DataFrame({'fala': [texto_teste], 'comportamento': [None]}))
        if len(textos) > 0:
            predicoes = modelo.predict(textos)
            print("Classificações:", predicoes)
        else:
            print("Nenhuma amostra disponível para avaliação.")
    else:
        print("Nenhum texto disponível para avaliação.")

# URLs dos arquivos de treinamento, teste e validação
url_treinamento = 'https://raw.githubusercontent.com/jbetha/mba/main/Analise_Sentimento_Assedio_dados_treinamento.csv'
url_teste = 'https://raw.githubusercontent.com/jbetha/mba/main/Alinhamento%20-%20Solu%C3%A7%C3%B5es%20T%C3%A9cnicas%20Transcript-1_23082024.txt'
url_validacao = 'https://raw.githubusercontent.com/jbetha/mba/main/Alinhamento%20de%20atividades%20Transcript_16082024.txt'

# Ler o arquivo CSV de treinamento
df_treinamento = ler_csv_url(url_treinamento)

# Preparar dados para treinamento a partir do CSV
textos_treinamento, etiquetas_treinamento = preparar_dados_csv(df_treinamento)

# Verificar se há dados suficientes para treinamento
if len(textos_treinamento) > 0:
    # Treinar o modelo de classificação
    modelo_comportamentos = treinar_modelo(textos_treinamento, etiquetas_treinamento)

    # Avaliar o modelo com o conjunto de teste
    print("Avaliação do Modelo com o Conjunto de Teste:")
    avaliar_modelo(modelo_comportamentos, url_teste)

    # Avaliar o modelo com o conjunto de validação
    print("Avaliação do Modelo com o Conjunto de Validação:")
    avaliar_modelo(modelo_comportamentos, url_validacao)
else:
    print("Não há dados suficientes para treinamento do modelo.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Número de amostras para treinamento: 22
Acurácia: 0.4
Precisão: 0.16
Recall: 0.4
F1-score: 0.2285714285714286
Avaliação do Modelo com o Conjunto de Teste:
Número de amostras para treinamento: 1
Classificações: ['desvalorização']
Avaliação do Modelo com o Conjunto de Validação:
Número de amostras para treinamento: 1
Classificações: ['desvalorização']


Para melhorar o modelo com base nos seus objetivos, aqui está um plano de ação:

# 1. Pré-processamento e Limpeza de Texto:

*   Limpeza: Remover stopwords, pontuação, números e converter texto para minúsculas.
*   Normalização: Remover acentuação e realizar stemming/lemmatização.
*   Balanceamento: Uso de técnicas de balanceamento para lidar com classes desbalanceadas, como Random Oversampling ou SMOTE (Synthetic Minority Over-sampling Technique).

# 2. Ajustes de Hiperparâmetros:

*   TfidfVectorizer: ngram_range: Ajustar para considerar combinações de palavras (bi-gramas ou tri-gramas).
*   max_df e min_df: Remover termos muito frequentes ou muito raros.
*   max_features: Limitar o número de termos para evitar sobreajuste.
*   MultinomialNB: Ajuste da suavização Laplace (alpha).

# 3. Validação Cruzada e Análise de Erros:
*   Implementar validação cruzada com K-fold para avaliar o desempenho do modelo.
*   Detalhar erros de classificação para entender onde o modelo falha.

In [2]:
# Instalar bibliotecas necessárias
!pip install nltk textblob scikit-learn pandas imbalanced-learn unidecode

# Importar bibliotecas
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.utils import resample
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
import pandas as pd
import re
import unidecode

# Baixar stopwords do nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Função de pré-processamento de texto
def preprocess_text(text):
    text = text.lower()  # Converte para minúsculas
    text = unidecode.unidecode(text)  # Remove acentuação
    text = re.sub(r'\d+', '', text)  # Remove números
    text = re.sub(r'[^\w\s]', '', text)  # Remove pontuação
    stop_words = set(stopwords.words('portuguese'))  # Carrega stopwords em português
    tokens = [word for word in text.split() if word not in stop_words]  # Remove stopwords
    return " ".join(tokens)

# Função para balancear as classes
def balancear_dados(X, y):
    X = X.values.reshape(-1, 1)  # Transformar X em um array 2D
    ros = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = ros.fit_resample(X, y)

    # Convertendo de volta para uma lista 1D
    X_resampled = [x[0] for x in X_resampled]

    print(f"Classes balanceadas: {Counter(y_resampled)}")
    return X_resampled, y_resampled

# Função para treinar o modelo de classificação com validação cruzada
def treinar_modelo_cv(textos, etiquetas):
    # Ajuste do TfidfVectorizer e MultinomialNB
    vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_df=0.9, min_df=5, max_features=5000)
    model = make_pipeline(vectorizer, MultinomialNB(alpha=0.1))

    # Validação cruzada
    y_pred = cross_val_predict(model, textos, etiquetas, cv=5)

    # Análise de desempenho
    print("Relatório de Classificação:")
    print(classification_report(etiquetas, y_pred))
    print("Matriz de Confusão:")
    print(confusion_matrix(etiquetas, y_pred))

    return model

# URL dos dados de treinamento
url_treinamento = 'https://raw.githubusercontent.com/jbetha/mba/refs/heads/main/Analise_Sentimento_Assedio_dados_treinamento.csv'

# Ler o arquivo CSV
df_treinamento = pd.read_csv(url_treinamento)

# Verificar colunas necessárias e pré-processar os textos
if 'fala' not in df_treinamento.columns or 'comportamento' not in df_treinamento.columns:
    raise ValueError("O arquivo CSV não contém as colunas 'fala' e 'comportamento'.")
df_treinamento['fala_limpa'] = df_treinamento['fala'].apply(preprocess_text)

# Balancear os dados
textos, etiquetas = balancear_dados(df_treinamento['fala_limpa'], df_treinamento['comportamento'])

# Treinar o modelo com validação cruzada
modelo = treinar_modelo_cv(textos, etiquetas)


Classes balanceadas: Counter({'desvalorização': 11, 'vida_privada': 11, 'delegação_inadequada': 11})
Relatório de Classificação:
                      precision    recall  f1-score   support

delegação_inadequada       0.57      0.36      0.44        11
      desvalorização       0.56      0.82      0.67        11
        vida_privada       0.90      0.82      0.86        11

            accuracy                           0.67        33
           macro avg       0.68      0.67      0.66        33
        weighted avg       0.68      0.67      0.66        33

Matriz de Confusão:
[[4 6 1]
 [2 9 0]
 [1 1 9]]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 1. Acurácia Geral
A acurácia geral do modelo é 67%, o que significa que o modelo está classificando corretamente 67% das amostras. Isso indica uma melhora em relação aos resultados anteriores, porém ainda pode haver espaço para aprimoramentos, especialmente no balanceamento das classes e na melhoria da generalização do modelo.

# 2. Análise por Classe:
**delegação inadequada:**

*  Precisão: 0.57 (57%) Dos exemplos classificados como "delegação inadequada", apenas 57% estavam corretos. Isso indica que o modelo está fazendo uma quantidade razoável de classificações erradas para esta classe.
*  Recall: 0.36 (36%) Apenas 36% dos exemplos reais de "delegação inadequada" foram identificados corretamente pelo modelo. Isso é um sinal de que o modelo está tendo dificuldades em identificar essa classe de forma consistente.
*  F1-score: 0.44 O F1-score médio (harmonização entre precisão e recall) é 44%, o que mostra que essa classe ainda é a mais desafiadora para o modelo.

**desvalorização:**

*  Precisão: 0.56 (56%) 56% dos exemplos classificados como "desvalorização" estavam corretos. Este valor é moderado.
*  Recall: 0.82 (82%) O modelo é muito bom em identificar a classe "desvalorização", com 82% dos exemplos reais sendo corretamente classificados.
*  F1-score: 0.67 A classe "desvalorização" apresenta um equilíbrio melhor, com um F1-score de 67%, indicando que o modelo lida razoavelmente bem com essa classe.

**vida privada:**

*  Precisão: 0.90 (90%) O modelo tem uma precisão alta ao classificar a classe "vida privada", com 90% das classificações para essa classe sendo corretas.
*  Recall: 0.82 (82%) O recall é também alto, indicando que a maioria dos exemplos reais de "vida privada" são corretamente identificados.
*  F1-score: 0.86 O F1-score de 86% mostra que o modelo se sai muito bem com essa classe, tanto em termos de precisão quanto de recall.

# 3. Matriz de Confusão:
A matriz de confusão apresenta a distribuição dos erros e acertos do modelo para cada classe:

[[4 6 1]  -> delegação inadequada
 [2 9 0]  -> desvalorização
 [1 1 9]] -> vida privada

**delegação inadequada:**
*  4 exemplos foram classificados corretamente.
*  6 exemplos de "delegação inadequada" foram erroneamente classificados como "desvalorização".
*  1 exemplo foi classificado incorretamente como "vida privada".

**desvalorização:**
*  9 exemplos foram classificados corretamente.
*  2 exemplos de "desvalorização" foram erroneamente classificados como "delegação inadequada".

**vida privada:**
*  9 exemplos foram classificados corretamente.
*  1 exemplo de "vida privada" foi classificado como "delegação inadequada".
*  1 exemplo foi erroneamente classificado como "desvalorização".

# 4. Sugestões para Melhorias:
* Classe "delegação inadequada": O modelo tem maior dificuldade com essa classe, com baixo recall (36%) e muitos exemplos erroneamente classificados como "desvalorização". **Uma possível melhoria seria aumentar o número de amostras para essa classe ou aplicar técnicas de aumento de dados.**

* Classe "desvalorização": Embora o recall seja alto, a precisão pode ser melhorada. O modelo pode estar classificando exemplos incorretamente como "desvalorização", sugerindo um possível desequilíbrio ou confusão entre classes.

* Classe "vida privada": Esta é a classe mais fácil para o modelo, com alto F1-score e poucos erros. O modelo lida bem com essa categoria.

In [6]:
# Importar bibliotecas
import nltk
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from imblearn.over_sampling import RandomOverSampler
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Baixar dados de análise de sentimentos do nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Função para ler o arquivo CSV de treinamento a partir de uma URL
def ler_csv_url(url):
    df = pd.read_csv(url)
    return df

# Função para pré-processamento do texto: limpeza, remoção de stopwords e lematização
def preprocessamento_texto(textos):
    stop_words = set(stopwords.words('portuguese'))
    lemmatizer = WordNetLemmatizer()

    textos_limpos = []
    for texto in textos:
        # Tokenizar e remover pontuação
        palavras = nltk.word_tokenize(texto.lower())
        palavras = [palavra for palavra in palavras if palavra.isalpha()]  # Mantém apenas palavras

        # Remover stopwords e aplicar lematização
        palavras = [lemmatizer.lemmatize(palavra) for palavra in palavras if palavra not in stop_words]
        textos_limpos.append(" ".join(palavras))

    return textos_limpos

# Função para balancear os dados com RandomOverSampler
def balancear_dados(X_train, y_train):
    ros = RandomOverSampler(random_state=42)
    X_res, y_res = ros.fit_resample(X_train, y_train)
    return X_res, y_res

# Função para treinar o modelo com validação cruzada e ajuste de hiperparâmetros
def treinar_modelo(textos_treinamento, etiquetas_treinamento):
    # Dividir dados em treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(textos_treinamento, etiquetas_treinamento, test_size=0.2, random_state=42)

    # Vetorizar o texto
    vectorizer = TfidfVectorizer(max_df=0.8, ngram_range=(1, 2))
    X_train_vect = vectorizer.fit_transform(X_train)
    X_test_vect = vectorizer.transform(X_test)

    # Balancear os dados usando RandomOverSampler
    X_train_bal, y_train_bal = balancear_dados(X_train_vect, y_train)

    # Treinar o modelo MultinomialNB
    modelo = MultinomialNB(alpha=0.1)
    modelo.fit(X_train_bal, y_train_bal)

    # Avaliar o modelo
    y_pred = modelo.predict(X_test_vect)
    print("Acurácia:", metrics.accuracy_score(y_test, y_pred))
    print("Precisão:", metrics.precision_score(y_test, y_pred, average='weighted'))
    print("Recall:", metrics.recall_score(y_test, y_pred, average='weighted'))
    print("F1-score:", metrics.f1_score(y_test, y_pred, average='weighted'))

    # Matriz de confusão
    print("Matriz de Confusão:")
    print(metrics.confusion_matrix(y_test, y_pred))

    return modelo

# URLs dos arquivos de treinamento
url_treinamento = 'https://raw.githubusercontent.com/jbetha/mba/refs/heads/main/Analise_Sentimento_Assedio_dados_treinamento.csv'

# Ler o arquivo CSV de treinamento
df_treinamento = ler_csv_url(url_treinamento)

# Pré-processar os textos de treinamento
df_treinamento['fala_limpa'] = preprocessamento_texto(df_treinamento['fala'])

# Treinar o modelo com validação cruzada e ajuste de hiperparâmetros
modelo_comportamentos = treinar_modelo(df_treinamento['fala_limpa'], df_treinamento['comportamento'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Acurácia: 0.8
Precisão: 0.6666666666666666
Recall: 0.8
F1-score: 0.72
Matriz de Confusão:
[[2 0 0]
 [0 2 0]
 [0 1 0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Os resultados mostram que o modelo está apresentando um desempenho bastante sólido, especialmente considerando a acurácia e o recall:

**Acurácia: 0.8** - Isso indica que o modelo está acertando 80% das previsões em relação ao total de amostras. É um bom sinal de que o modelo tem um desempenho razoável na classificação geral.

**Precisão: 0.67** - A precisão, que mede a proporção de verdadeiros positivos entre todas as previsões positivas, sugere que, embora o modelo tenha uma boa taxa de acertos, ele comete erros ao classificar algumas amostras.

**Recall: 0.8** - Isso mostra que o modelo é capaz de identificar 80% das instâncias reais da classe positiva. Um recall alto é positivo, especialmente em contextos onde perder um caso positivo é crítico.

**F1-score: 0.72** - Este valor é uma média harmônica da precisão e do recall, e um F1-score de 0.72 é um bom indicador de que o modelo está equilibrando bem entre a precisão e o recall.

**A matriz de confusão indica que:**
* Para a **classe 0** (talvez "delegação inadequada"), o modelo acertou todas as previsões (2 acertos, 0 erros).
* Para a **classe 1** (talvez "desvalorização"), o modelo também acertou todas as previsões (2 acertos, 0 erros).
* Para a **classe 2** (talvez "vida privada"), o modelo identificou corretamente uma amostra, mas confundiu uma amostra, classificando-a como pertencente à classe 1.

# Sugestões para Melhoria
**Aumento do Conjunto de Dados:** Se possível, aumentar a quantidade de dados de treinamento pode ajudar a melhorar o desempenho geral do modelo, especialmente em classes menos representadas.

**Ajuste Fino dos Hiperparâmetros:** Continuar a explorar ajustes nos hiperparâmetros do MultinomialNB e do TfidfVectorizer, como o ajuste da regularização (alpha), o número de n-gramas, ou até mesmo testar outros vetorizadores, como o CountVectorizer.

**Análise de Erros:** Examinar os casos que foram classificados incorretamente para entender por que isso aconteceu. Isso pode fornecer insights sobre o que melhorar, seja através de mais dados, mais características de entrada ou ajustando o modelo.

**Exploração de Outros Modelos:** Considerar a aplicação de outros modelos de aprendizado de máquina, como SVM, árvores de decisão ou redes neurais, que podem capturar padrões diferentes nos dados.

**Validação Cruzada:** Implementar validação cruzada para garantir que o modelo seja robusto e não apenas esteja se ajustando bem a um conjunto específico de dados.

In [8]:
# Importar bibliotecas
import nltk
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from imblearn.over_sampling import RandomOverSampler
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Baixar dados de análise de sentimentos do nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Função para ler o arquivo CSV de treinamento a partir de uma URL
def ler_csv_url(url):
    df = pd.read_csv(url)
    return df

# Função para pré-processamento do texto: limpeza, remoção de stopwords e lematização
def preprocessamento_texto(textos):
    stop_words = set(stopwords.words('portuguese'))
    lemmatizer = WordNetLemmatizer()

    textos_limpos = []
    for texto in textos:
        # Tokenizar e remover pontuação
        palavras = nltk.word_tokenize(texto.lower())
        palavras = [palavra for palavra in palavras if palavra.isalpha()]  # Mantém apenas palavras

        # Remover stopwords e aplicar lematização
        palavras = [lemmatizer.lemmatize(palavra) for palavra in palavras if palavra not in stop_words]
        textos_limpos.append(" ".join(palavras))

    return textos_limpos

# Função para balancear os dados com RandomOverSampler
def balancear_dados(X_train, y_train):
    ros = RandomOverSampler(random_state=42)
    X_res, y_res = ros.fit_resample(X_train, y_train)
    return X_res, y_res

# Função para treinar o modelo com validação cruzada e ajuste de hiperparâmetros
def treinar_modelo(textos_treinamento, etiquetas_treinamento):
    # Dividir dados em treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(textos_treinamento, etiquetas_treinamento, test_size=0.2, random_state=42)

    # Vetorizar o texto
    vectorizer = TfidfVectorizer(max_df=0.8, ngram_range=(1, 2))
    X_train_vect = vectorizer.fit_transform(X_train)
    X_test_vect = vectorizer.transform(X_test)

    # Balancear os dados usando RandomOverSampler
    X_train_bal, y_train_bal = balancear_dados(X_train_vect, y_train)

    # Treinar o modelo MultinomialNB
    modelo = MultinomialNB()

    # Ajuste fino dos hiperparâmetros usando GridSearchCV
    parametros = {'alpha': [0.01, 0.1, 1.0, 10.0]}
    grid_search = GridSearchCV(modelo, parametros, cv=5, scoring='f1_weighted')
    grid_search.fit(X_train_bal, y_train_bal)

    print("Melhores hiperparâmetros:", grid_search.best_params_)
    modelo = grid_search.best_estimator_

    # Avaliar o modelo com validação cruzada
    scores = cross_val_score(modelo, X_train_bal, y_train_bal, cv=5, scoring='f1_weighted')
    print("F1-score médio com validação cruzada:", np.mean(scores))

    # Treinar o modelo final
    modelo.fit(X_train_bal, y_train_bal)

    # Avaliar o modelo no conjunto de teste
    y_pred = modelo.predict(X_test_vect)
    print("Acurácia:", metrics.accuracy_score(y_test, y_pred))
    print("Precisão:", metrics.precision_score(y_test, y_pred, average='weighted'))
    print("Recall:", metrics.recall_score(y_test, y_pred, average='weighted'))
    print("F1-score:", metrics.f1_score(y_test, y_pred, average='weighted'))

    # Matriz de confusão
    print("Matriz de Confusão:")
    cm = metrics.confusion_matrix(y_test, y_pred)
    print(cm)

    # Análise de erros
    erros = pd.DataFrame({'fala': X_test, 'etiqueta_real': y_test, 'etiqueta_predita': y_pred})
    erros = erros[erros['etiqueta_real'] != erros['etiqueta_predita']]
    print("Análise de erros:")
    print(erros)

    # Relatório detalhado de métricas por classe
    print("Relatório de Classificação:")
    print(metrics.classification_report(y_test, y_pred))

    return modelo

# URLs dos arquivos de treinamento
url_treinamento = 'https://raw.githubusercontent.com/jbetha/mba/refs/heads/main/Analise_Sentimento_Assedio_dados_treinamento.csv'

# Ler o arquivo CSV de treinamento
df_treinamento = ler_csv_url(url_treinamento)

# Pré-processar os textos de treinamento
df_treinamento['fala_limpa'] = preprocessamento_texto(df_treinamento['fala'])

# Treinar o modelo com validação cruzada e ajuste de hiperparâmetros
modelo_comportamentos = treinar_modelo(df_treinamento['fala_limpa'], df_treinamento['comportamento'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Melhores hiperparâmetros: {'alpha': 0.01}
F1-score médio com validação cruzada: 0.8806677843724904
Acurácia: 0.6578947368421053
Precisão: 0.7033492822966507
Recall: 0.6578947368421053
F1-score: 0.6583333333333334
Matriz de Confusão:
[[ 0  0  0  0  0  0]
 [ 1 14  3  0  0  0]
 [ 0  6  7  0  0  0]
 [ 0  0  0  2  0  0]
 [ 0  1  0  0  1  0]
 [ 0  1  1  0  0  1]]
Análise de erros:
                                                  fala         etiqueta_real  \
175  priscila lourenço molina bastante buraco jagua...  delegação_inadequada   
65   jaguaraci batista silva falei pra agnes consig...        desvalorização   
141  priscila lourenço molina assim fica ficando cl...        desvalorização   
124  jaguaraci batista silva loga avd própria rede ...            isolamento   
164  priscila lourenço molina desconectou né ah lem...  delegação_inadequada   
120  priscila lourenço molina normalmente têm bloqu...          vida_privada   
66   priscila lourenço molina acho jaguar reuniões ...  delega

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarni